## Embedding model 의 문장 유사도 테스트
### [KoSimCSE-roberta](https://huggingface.co/BM-K/KoSimCSE-roberta)

In [20]:
import torch
from transformers import AutoModel, AutoTokenizer

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')

In [21]:
def embedding_similarity(model, tokenizer, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    embeddings, _ = model(**inputs, return_dict=False)

    score01 = cal_score(embeddings[0][0], embeddings[1][0])
    score02 = cal_score(embeddings[0][0], embeddings[2][0])
    score03 = cal_score(embeddings[1][0], embeddings[2][0])
    
    score = [score01, score02, score03 ]
    max_score = max(score).item()
    max_score_idx = score.index(max(score))
    print(score01.item(), score02.item(), score03.item())
    if max_score_idx == 0:
        print(f"1,2번 문장이 {max_score}로 가장 유사합니다.")
    elif max_score_idx == 1:  
        print(f"1,3번 문장이 {max_score}로 가장 유사합니다.")
    else:    
        print(f"2,3번 문장이 {max_score}로 가장 유사합니다.") 

    return 

In [22]:
sentences = ['치타가 들판을 가로 질러 먹이를 쫓는다.',
             '치타 한 마리가 먹이 뒤에서 달리고 있다.',
             '원숭이 한 마리가 드럼을 연주한다.']
embedding_similarity(model, tokenizer, sentences)

73.72628021240234 18.873348236083984 17.086091995239258
1,2번 문장이 73.72628021240234로 가장 유사합니다.
